In [1]:
from web3 import Web3
from solcx import compile_source
import pandas as pd
from datetime import datetime

In [2]:
URL="HTTP://127.0.0.1:7545"

In [15]:
def connect(URL):
    web3=Web3(Web3.HTTPProvider(URL))
    print("Connected?:",web3.isConnected())
    return web3

def get_user_accounts():
    return web3.eth.accounts

def create_default_account(account_hash):
    web3.eth.default_account=account_hash
    
def compile_solidity(file):
    with open(file,'r') as f:
        source=f.read()
    return compile_source(source)

def get_abi_bytecode(compiled_sol):
    contract_id,contract_abi=compiled_sol.popitem()
    abi=contract_abi['abi']
    byte_code=contract_abi['bin']
    return abi,byte_code

def deploy_contract(abi,byte_code):
    contract=web3.eth.contract(abi=abi,bytecode=byte_code)
    tx_hash=contract.constructor().transact()
    tx_receipt=web3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receipt

web3=connect(URL)
print(get_user_accounts())

Connected?: True
['0x4cbc87586da52F809D98F122331C06Ba16D6cC3A', '0xA4a4a85a3aA829384e2de7aD2B16B06EBA07A2bA', '0x58325180563bC75A12543FD380dE90048b89700c', '0xB1ec37fA914Fcf087BC1FB394F53292f337150Be', '0x2ad1fD7BD41DBA854523ecf0E3fbef6E7fC91af1', '0x7107246e4d83cF65F0Fbc56584b274B68745f40d', '0xb40518c06eF3dad45D3F35D3e839cF9b4A230f97', '0x464244D4cfDcE7aB54694dbBcd6C3E1ddCAb8c0a', '0x9108B087b51E0fE9Fa9D37eb794386A98050Bf48', '0x426658F34b041dA97E78ae4972Af6f472b1363e4']


In [16]:
create_default_account('0x4cbc87586da52F809D98F122331C06Ba16D6cC3A')

In [18]:
compiled_sol=compile_solidity('contract.sol')

In [19]:
abi,byte_code=get_abi_bytecode(compiled_sol)

In [20]:
abi

[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'inputs': [],
  'name': 'amount',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'getUser',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'},
   {'internalType': 'uint256', 'name': '', 'type': 'uint256'},
   {'internalType': 'address', 'name': '', 'type': 'address'},
   {'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256',
    'name': 'invoiceAmount',
    'type': 'uint256'},
   {'internalType': 'address', 'name': 'to', 'type': 'address'}],
  'name': 'payAmount',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [],
  'name': 'payme',
  'outputs': [],
  'stateMutability': 'payable',
  'type': 'function'}]

In [21]:
byte_code

'608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610475806100606000396000f3fe60806040526004361061003f5760003560e01c8063832880e71461004457806396bc678514610072578063aa8c217c1461009b578063ba6361fb146100c6575b600080fd5b34801561005057600080fd5b506100596100d0565b604051610069949392919061030b565b60405180910390f35b34801561007e57600080fd5b5061009960048036038101906100949190610296565b61013b565b005b3480156100a757600080fd5b506100b061024b565b6040516100bd91906102f0565b60405180910390f35b6100ce610251565b005b6000806000806001600001546001800154600160020160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16600160030160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16935093509350935090919293565b600042905060008390506000839050600033905060405180608001604052808581526020018481526020018273ffffffffffffffffffffffffffffffffffffffff1681526020018373fffffffffffffffffffffffffffff

In [22]:
receipt=deploy_contract(abi=abi,byte_code=byte_code)

In [23]:
receipt

AttributeDict({'transactionHash': HexBytes('0x8d346093775c0d42d974837faf8dc9970f7f0b330b92d4365308b14d9c99d7a8'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xc3fc06fefb1cc8047bb23c6f2f3866c05b36403ee34642d8259cb692527342d5'),
 'blockNumber': 46,
 'from': '0x4cbc87586da52F809D98F122331C06Ba16D6cC3A',
 'to': None,
 'gasUsed': 320987,
 'cumulativeGasUsed': 320987,
 'contractAddress': '0xf0632C92e6FA2F1331fCc098B37d46e673170A88',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [24]:
contract=web3.eth.contract(address=receipt['contractAddress'],abi=abi)

In [25]:
contract.functions.payAmount(100,"0x58325180563bC75A12543FD380dE90048b89700c").transact()

HexBytes('0x91d51d3c548a3a182c156695ef4016b9cb0369883d6a7a56c0f076ce0146c375')

In [26]:
data=contract.functions.getUser().call()
data

[1617806949,
 100,
 '0x4cbc87586da52F809D98F122331C06Ba16D6cC3A',
 '0x58325180563bC75A12543FD380dE90048b89700c']

In [27]:
datetime_date=datetime.fromtimestamp(data[0])
pd.to_datetime(datetime_date)

Timestamp('2021-04-07 20:34:09')

In [31]:
contract.functions.amount().call()

0